In [41]:
import tensorflow as tf
from tensorflow.keras import layers, models

img_width = 25
img_height = 50
batch_size = 50

# Define the directory containing the images
directory = r'C:\Users\SRIVATSAL NARAYAN\Desktop\scoliosis-detection-main\Spine X-Rays\ImagesOriginalSize'

# Define the classes
class_names = ['NormalFinal', 'ScolFinal', 'SpondFinal']

# Load the dataset from directory
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode="int",
    class_names=class_names,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='training'
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode="int",
    class_names=class_names,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='validation'
)

# Define the model
model = models.Sequential()
model.add(layers.Conv2D(50, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(100, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(100, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(len(class_names)))

# Compile and train the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(ds_train, epochs=20, verbose=2)

# Save the trained model
spine_rec_model = 'spine_rec_model.pk1'
model.save(spine_rec_model)


Found 338 files belonging to 3 classes.
Using 305 files for training.
Found 338 files belonging to 3 classes.
Using 33 files for validation.
Epoch 1/20
7/7 - 2s - loss: 13.1201 - accuracy: 0.4557 - 2s/epoch - 321ms/step
Epoch 2/20
7/7 - 1s - loss: 1.1287 - accuracy: 0.5180 - 848ms/epoch - 121ms/step
Epoch 3/20
7/7 - 1s - loss: 0.7891 - accuracy: 0.6525 - 837ms/epoch - 120ms/step
Epoch 4/20
7/7 - 1s - loss: 0.5862 - accuracy: 0.7443 - 846ms/epoch - 121ms/step
Epoch 5/20
7/7 - 1s - loss: 0.4699 - accuracy: 0.7869 - 934ms/epoch - 133ms/step
Epoch 6/20
7/7 - 1s - loss: 0.3318 - accuracy: 0.9016 - 790ms/epoch - 113ms/step
Epoch 7/20
7/7 - 1s - loss: 0.2336 - accuracy: 0.9148 - 799ms/epoch - 114ms/step
Epoch 8/20
7/7 - 1s - loss: 0.2727 - accuracy: 0.9082 - 841ms/epoch - 120ms/step
Epoch 9/20
7/7 - 1s - loss: 0.2465 - accuracy: 0.8984 - 795ms/epoch - 114ms/step
Epoch 10/20
7/7 - 1s - loss: 0.1755 - accuracy: 0.9475 - 847ms/epoch - 121ms/step
Epoch 11/20
7/7 - 1s - loss: 0.1681 - accuracy: 0.

INFO:tensorflow:Assets written to: spine_rec_model.pk1\assets


In [43]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained model
spine_rec_model = 'spine_rec_model.pk1'
model = load_model(spine_rec_model)

# Define the directory containing the example images
example_directory = r'C:\Users\SRIVATSAL NARAYAN\Desktop\scoliosis-detection-main\Spine X-Rays\ImagesOriginalSize'

# Load a few example images from the directory
example_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
example_data = example_generator.flow_from_directory(
    example_directory,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='sparse',
    color_mode='grayscale',  # Ensure grayscale mode
    shuffle=True
)

# Get class labels
class_labels = {v: k for k, v in example_data.class_indices.items()}

# Make predictions on example images
for i in range(3):  # Change the range to the number of examples you want to predict
    image, label = next(example_data)
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)
    actual_class = int(label[0])
    print("Example", i+1)
    print("Predicted class:", class_labels[predicted_class])
    print("Actual class:", class_labels[actual_class])
    print()


Found 338 images belonging to 3 classes.
1/1 [==============================] - 0s 89ms/step
Example 1
Predicted class: SpondFinal
Actual class: SpondFinal

1/1 [==============================] - 0s 23ms/step
Example 2
Predicted class: SpondFinal
Actual class: ScolFinal

1/1 [==============================] - 0s 21ms/step
Example 3
Predicted class: ScolFinal
Actual class: ScolFinal

